In [ ]:
#default_exp training

In [ ]:
#export
from speechsep import *
from speechsep.data import *
from speechsep.imports import *
from speechsep.models import *

from torch.utils.data import Dataset, DataLoader
import torch
import torchvision

In [ ]:
#export
fn = Path("../data/esc50_sample/")
pipe = AudioPipe(fn)

# Dataset/DataLoader

In [ ]:
#export
class AudioDataset(Dataset):
    @delegates(AudioPipe)
    def __init__(self, fn, **kwargs):
        self.fn = fn
        self.pipe = AudioPipe(fn)
        self.n_samples = len(get_audio_files(fn))
    
    def __getitem__(self, index):
        x,y = self.pipe(index)
        x,y = ToTensor()(x)[:,:208],ToTensor()(y) #Hack add multiple or padding to pipeline
        return x,y
    
    def __len__(self):
        return self.n_samples

In [ ]:
#export
dataset = AudioDataset(fn)
dataloader = DataLoader(dataset=dataset, batch_size=4, shuffle=True, num_workers=2)

dataiter = iter(dataloader)

In [ ]:
#export
data = dataiter.next()

Tensorify = ToTensor()

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/holydemon/.local/anaconda3/envs/fai2/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/holydemon/.local/anaconda3/envs/fai2/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 47, in fetch
    return self.collate_fn(data)
  File "/home/holydemon/.local/anaconda3/envs/fai2/lib/python3.7/site-packages/torch/utils/data/_utils/collate.py", line 79, in default_collate
    return [default_collate(samples) for samples in transposed]
  File "/home/holydemon/.local/anaconda3/envs/fai2/lib/python3.7/site-packages/torch/utils/data/_utils/collate.py", line 79, in <listcomp>
    return [default_collate(samples) for samples in transposed]
  File "/home/holydemon/.local/anaconda3/envs/fai2/lib/python3.7/site-packages/torch/utils/data/_utils/collate.py", line 81, in default_collate
   

# Training Loop

In [ ]:
#export
n_epochs = 1
n_samples = len(dataset)
n_iter = math.ceil(n_samples/4)

In [ ]:
#export
model = U_Net(img_ch=2, output_ch=4)
model.train();

In [ ]:
#export
for epoch in range(n_epochs):
    for i, (mixed, specs) in enumerate(dataloader):
        xb,yb = Tensorify(mixed), Tensorify(specs)
        if (i+1)%5==0:
            print(f'epoch {epoch}: step {(i+1)/n_iter}')
        out = model(xb)
        mask1 = MaskcIRM(out[:,:2,:,:])
        mask2 = MaskcIRM(out[:,2:,:,:])
        sepspecs = mask1*mixed, mask2*mixed
        loss = loss_func(out, yb)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()